In [65]:
import rasterio
import geopandas as gpd
import numpy as np
from skimage.graph.mcp import MCP_Geometric
from onstove.raster import *

In [66]:
# This map represents the travel speed from this allocation process, expressed in units of minutes 
# required to travel one metre.
friction = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\new_friction.tif"
# Startpoints
supply_points = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\LPG_start.shp"
gdf_supply_points = gpd.read_file(supply_points)
#distance raster
distance = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\distance.tif"
#new friction based on start points
travel_time = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\travel_time_2.tif"

In [67]:
# Read raster as array and set nodata to infinity to avoid taking nodata cells as path
with rasterio.open(friction) as src:
    fric = src.read(1)
    fric = fric*1000/60

In [68]:
# Minimum cost path betweenn all nodes
mcp = MCP_Geometric(fric, fully_connected=True)

In [69]:
#Finding start points
row, col = friction_start_points(friction, supply_points, travel_time)

In [70]:
#Adding statvalues
pointlist = []
i = 0
for test in row:
    positiions = [row[i],col[i]]
    pointlist.append(positiions)
    i = i+1

In [71]:
#minimum cost
cumulative_costs, traceback = mcp.find_costs(starts=pointlist)
cumulative_costs[np.where(cumulative_costs==float('inf'))] = np.nan

In [72]:
new_dataset = rasterio.open(travel_time, 'w', driver='GTiff',
                            height = src.shape[0], width = src.shape[1],
                            count=1, dtype="float64",
                            crs=src.crs, transform = src.transform)

new_dataset.write(cumulative_costs, 1)
new_dataset.close()